**1. Business Understanding**

Airbnb is an online platform that provides home lodging services around the world. Understanding its market values, experiences and rights of host and guest, and safety concerns can be made through data exploration of the previous data.
3 business problems to be observed:
1.	What is the peak season in Seattle in a year?
2.	Which area is the crowdest in Seattle?
3.	Price prediction

**2. Data Understanding**

There are 3 datasets provided:
1.	Calendar, including listing id and price for the availability day.
2.	Listings, including listings information and its scores.
3.	Reviews, including reviewer information and their comments.

2 datasets - Calendar and Listings will be used to analyzed.

Import relevant libraries and required datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

calendar = pd.read_csv('/kaggle/input/seattle/calendar.csv')
listings = pd.read_csv('/kaggle/input/seattle/listings.csv')

In [ ]:
calendar.head()

In [ ]:
calendar.info()

In [ ]:
listings.head()

In [ ]:
listings.info()

**3. Data Preparation**

Convert data types into correct versions on calendar and listings datasets

In [ ]:
#Convert integer to string
calendar['listing_id'] = calendar['listing_id'].astype(str)

#Convert to datetime and extract month and year
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['month'] = pd.to_datetime(calendar['date']).dt.month

#Eliminate and convert to float
calendar['price'] = calendar['price'].replace('\$|,', '', regex=True).astype('float')

In [ ]:
calendar.head()

In [ ]:
#Create new datast for related features 
df = listings[['price','accommodates','bathrooms','bedrooms',
               'beds','weekly_price','monthly_price','cleaning_fee', 
               'instant_bookable','reviews_per_month','cancellation_policy']]
df.head()

From df dataset, there are incorrect data types and NaN values.

In [ ]:
#Convert data type string into float
df[['price', 'weekly_price', 'monthly_price', 'cleaning_fee']] = df.iloc[:, [0,5,6,7]].replace('\$|,', '', regex = True).astype('float')

df.head()

**Imputing Values in Numerical Variable**

NaN values in the data frame can be imputed with mean value of the columns to prevent dropping of rows or columns. 

In [ ]:
#Impute values 
num_vars = df.select_dtypes(include=['int', 'float']).columns
for num in num_vars:
    df[num].fillna(df[num].mean(), inplace = True)

**Create dummies to NaN values in Categorical Variable**

 Categorical variable column break into 1 or 0 groups and summarize into column dummy_na.

In [ ]:
#Break NaN values into dummy coded
cat_vars = df.select_dtypes(include=['object']).columns
for cat in cat_vars:
    df = pd.concat([df.drop(cat, axis=1), pd.get_dummies(df[cat], prefix = cat, prefix_sep = '_', drop_first = True)],  axis = 1)

In [ ]:
#Check for null
np.sum(df.isnull())

NaN values have been fully replaced.

**4. Modelling & Evaluation**

Questions 1: What is the peak season in Seattle in a year??

In [ ]:
#Extract count of 'f' from available's column
calendar_f = pd.pivot_table(calendar[calendar['available'] == 'f'], index= 'month', aggfunc = {'available': 'count'})
calendar_f['month'] = calendar_f.index

#Extract count of 't' from available's column
calendar_t = pd.pivot_table(calendar[calendar['available'] == 't'], index= 'month', aggfunc = {'available': 'count'})
calendar_t['month'] = calendar_t.index

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows = 1, ncols = 2, sharey = True, figsize = (10, 4))
f = sns.barplot(x = calendar_f['month'], y = calendar_f['available'], ax = ax0)
t = sns.barplot(x = calendar_t['month'], y = calendar_t['available'], ax = ax1)
ax0.set(xlabel = 'Month', ylabel = 'Count of f', xlim = (-1, 12))
ax1.set(xlabel = 'Month', ylabel = 'Count of t', xlim = (-1, 12))
f.set_title('Available - f', y = 1)
t.set_title('Available - t', y = 1)
plt.show()

From the graph of available f above, the peak season in Seattle was observed to fall in January, June, and August in a year. The availability of listings is higher in March and December.

**Question 2: Which area is the crowdest in Seattle?**

In [ ]:
listings['zipcode'] = listings['zipcode'].dropna(axis = 0).astype('category')
location = listings[['latitude', 'longitude', 'zipcode']]

In [ ]:
sns.scatterplot(data = location, x = 'longitude', y = 'latitude', hue = 'zipcode').set(title = 'Location of Listings')
sns.set(rc = {'figure.figsize':(20,20)})
plt.xticks(rotation = 90)
plt.show()

From the scatter plot above, most of the listings are located at the center of Seattle with zip codes 98122, 98103, and 98102.

**Question 3: Price prediction**

Several features have been selected from listings dataset.

In [ ]:
df.head()

In [ ]:
#Overview the correlation of price
sns.heatmap(df.corr(), annot = True, linewidths = .3)
plt.figure(figsize = (5, 5))
plt.show()

In [ ]:
#Define X and y 
y = df['price']
df = df.drop('price', axis = 1)
X = df

In [ ]:
X.info()

In [ ]:
#Instantiate model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)
lm_model = LinearRegression(normalize = True)

#Fit the model
lm_model.fit(X_train, y_train)

#Predict fitted model on test data
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)
test_score = r2_score(y_test, y_test_preds)
train_score = r2_score(y_train, y_train_preds)

print('rsquared_score: {} \ntrain_score: {}'.format(test_score, train_score))

Based on the rsqueared score and train score, both data fits well.

In [ ]:
#Create dataframe to understand the most influential coefficients in a linear model
coef_df = pd.DataFrame()
coef_df['features'] = X_train.columns
coef_df['coef'] = lm_model.coef_
coef_df['abs_coef'] = np.abs(lm_model.coef_)
coef_df = coef_df.sort_values('abs_coef', ascending = False)
print('Weight of Features on Price:\n', coef_df, '\n')

In [ ]:
plt.bar(coef_df['features'], coef_df['abs_coef'])
plt.xlabel('Features')
plt.ylabel('abs_coef')
plt.title('Weight of Features on Price')
plt.xticks(coef_df['features'], rotation = 90)
plt.figure(figsize = (15,5))
plt.show()

Features of bathrooms, accommodation, and bedrooms show many contribution to the price.

**Summary**

From the analysis, the Airbnb business in Seattle reached its maximum in January, July, and August. Most of the listings are located at the center of Seattle. Price is defined by the bathrooms, accommodation, bedrooms of listings.